# MyJohnDeere API Best Practices

This notebook contains hints, best practices, and errata to help use the MyJohnDeere API's.

*Note - This notebook requires that you be logged into Google.*

Copyright (c) 2019 Deere & Company
 
This software may be modified and distributed under the terms
of the MIT license.  See the LICENSE file for details.

# Create MyJohnDeere API OAuth2 session

If you haven't already, make sure you have gone through the [MyJohnDeere Onboarding](https://colab.research.google.com/github/JohnDeere/DevelopWithDeereNotebooks/blob/master/Onboarding/myjohndeere-api-onboarding.ipynb) notebook to better understand how to setup an OAuth2 session.  The code below simply re-iterates the steps in that notebook to setup an OAuth2 session.  Here we go through those steps again to get a valid session to use for the remainder of this notebook. 

Import some utility libraries to assist with seting up and managing an OAuth session. You can find similar libraries for the programming language of your choice

In [1]:
from requests_oauthlib import OAuth2Session
import requests
import json

Specify your client ID, client secret, and client redirect URI below

In [2]:
CLIENT_ID = '0oa9u3fohqdKLFBbs5d7'
CLIENT_SECRET = 'rX3y-uDy5DteR640N2fJKBuFrohaI0-rF2kOwLs3'
CLIENT_REDIRECT_URI = 'http://loacalhost:9090/callback'

# Leave the line below as-is. This line of code verifies that you've modified the CLIENT_ID, CLIENT_SECRET, CLIENT_REDIRECT_URI to the values above so that your application can complete OAuth"
assert(CLIENT_ID != 'place_client_key_here' and CLIENT_SECRET != 'place_client_secret_here' and CLIENT_REDIRECT_URI != 'place_client_redirect_uri_here'), "You need to update your CLIENT_ID, CLIENT_SECRET, or CLIENT_REDIRECT_URI in this cell"

Query the well known OAuth URL to determine the OAuth authorization URL, token grant URL, and available scopes.

In [3]:
WELL_KNOWN_URL = 'https://signin.johndeere.com/oauth2/aus78tnlaysMraFhC1t7/.well-known/oauth-authorization-server'

# Query the ./well-known OAuth URL and parse out the authorization URL, the token grant URL, and the available scopes
well_known_response = requests.get(WELL_KNOWN_URL)
well_known_info = json.loads(well_known_response.text)

AUTHORIZATION_URL = well_known_info['authorization_endpoint']
TOKEN_GRANT_URL = well_known_info['token_endpoint']
AVAILABLE_SCOPES = str(' ').join(well_known_info['scopes_supported'])

print('Well Known Authorization URL - ' + AUTHORIZATION_URL)
print('Well Known Token Grant URL - ' + TOKEN_GRANT_URL)
print('Available Scopes - ' + AVAILABLE_SCOPES)

Well Known Authorization URL - https://signin.johndeere.com/oauth2/aus78tnlaysMraFhC1t7/v1/authorize
Well Known Token Grant URL - https://signin.johndeere.com/oauth2/aus78tnlaysMraFhC1t7/v1/token
Available Scopes - ag1 ag2 ag3 eq1 eq2 files finance1 finance2 org1 org2 work1 work2 openid profile email address phone offline_access device_sso


Request an authorization code

In [4]:
SCOPES_TO_REQUEST = {'offline_access', 'ag1', 'eq1', 'files'}
STATE = "Some Unique Identifier"
oauth2_session = OAuth2Session(CLIENT_ID,  redirect_uri=CLIENT_REDIRECT_URI, scope=SCOPES_TO_REQUEST)

authorization_request, state = oauth2_session.authorization_url(AUTHORIZATION_URL, STATE)
print("Click on the following link to present the user with sign in form where they authenticate and approve access to your application.")
print(authorization_request) 

Click on the following link to present the user with sign in form where they authenticate and approve access to your application.
https://signin.johndeere.com/oauth2/aus78tnlaysMraFhC1t7/v1/authorize?response_type=code&client_id=0oa9u3fohqdKLFBbs5d7&redirect_uri=http%3A%2F%2Floacalhost%3A9090%2Fcallback&scope=offline_access+files+eq1+ag1&state=Some+Unique+Identifier


After the user authenticates and approves accces, copy the authorization code from the redirect URL and paste it in the next code section")
![Verifier Screen](https://github.com/hutch4cy/DevelopWithDeereNotebooks/raw/master/Onboarding/OAuth2Code.jpg)

Request an access token

In [5]:
# Update the authorization code here
AUTHORIZATION_CODE = 'lmivnEmVb4Y94YV3dUm1Txfk8_LVak_IumowcKhAgpg'

# Leave the line below as-is. This is to make sure that you have update the AUTHORIZATION_CODE
assert(AUTHORIZATION_CODE != 'place_authorization_code_here'), 'The AUTHORIZATION_CODE in this cell must be replaced by the authorization_code that you recieved'

# Now that we have an authorization code, lets fetch an access and refresh token
token_response = oauth2_session.fetch_token(TOKEN_GRANT_URL, code=AUTHORIZATION_CODE, client_secret=CLIENT_SECRET)
access_token = token_response['access_token']
refresh_token = token_response['refresh_token']

# Also take note that the access token expiration time is returned.  When the access token expires, 
# you will want to use the refresh token to request a new access token (described later in this notebook)
access_token_expiration = token_response['expires_in']

print("Access Token: " + access_token)
print("Refresh Token: " + refresh_token)
print("Hours Token Is Valid: " + str(int(access_token_expiration/60/60)))

Access Token: eyJraWQiOiJNYy1mX1BROElHSFpVeHRKc3pKd0lUeHRHVjlLS081Q1J1Zm5ZbDZyN0xzIiwiYWxnIjoiUlMyNTYifQ.eyJ2ZXIiOjEsImp0aSI6IkFULnVpRk5zZlhVVXRlOWJhY2ZMQlJyeGs5ckt3TnpKS3FtaUgzVVZ4dXlCMU0ub2FyMTRlZnp6NmsxZkdHU2E1ZDciLCJpc3MiOiJodHRwczovL3NpZ25pbi5qb2huZGVlcmUuY29tL29hdXRoMi9hdXM3OHRubGF5c01yYUZoQzF0NyIsImF1ZCI6ImNvbS5kZWVyZS5pc2cuYXhpb20iLCJpYXQiOjE2ODY4MTczNjAsImV4cCI6MTY4Njg2MDU2MCwiY2lkIjoiMG9hOXUzZm9ocWRLTEZCYnM1ZDciLCJ1aWQiOiIwMHU5dTMyNW04NXBhWmY1YjVkNyIsInNjcCI6WyJhZzEiLCJvZmZsaW5lX2FjY2VzcyIsImZpbGVzIiwiZXExIl0sImF1dGhfdGltZSI6MTY4NjgxNjUzNywic3ViIjoiQ2hyaXN0aWFuS2F1IiwiaXNjc2MiOnRydWUsInRpZXIiOiJTQU5EQk9YIiwiY25hbWUiOiJFbXBsb3ltZW50IFRhc2siLCJ1c2VyVHlwZSI6IkN1c3RvbWVyIn0.f3oM6a89tRdLdamVJWfjKjGVnF4hV_6h-Dypw94DclkqXXDEctdjxARR8fEm4snUznDz2zjOfNDxzkEvpmOpafJW0WExhglNgb2YeZklLhnCU82G0YaFSGsN-IuT3dUlE13loG9KWmv6XS_Dm6Puo1T_ZIVEqV2vZ4udOZ8lFSWaNSiyA_E80HatTEoAkEqPZQYStzPlc9vq2NPjbITeULGAqJSQa2XQLrhxGhNj4JeJ4bsTQdm4yufAIznicE8IMJvpIAoqPDcKXCxlq3eV70UFewYO_2HzgbrP2pa58AYS6M2RFbJvkMw

Call the MyJohnDeere API Catalog to verify the OAuth session

In [6]:
MYJOHNDEERE_V3_JSON_HEADERS = { 'Accept': 'application/vnd.deere.axiom.v3+json',
                                'Content-Type': 'application/vnd.deere.axiom.v3+json'}

# If your app happens to be already approved for production, then use the partnerapi.deere.com, otherwise stick with sandboxapi.deere.com
API_CATALOG_URI = 'https://sandboxapi.deere.com/platform/'
#API_CATALOG_URI = 'https://partnerapi.deere.com/platform/'

api_catalog_response = oauth2_session.get(API_CATALOG_URI, headers=MYJOHNDEERE_V3_JSON_HEADERS) 

print("Request returned - " + str(api_catalog_response.status_code))
assert(200 == api_catalog_response.status_code)

Request returned - 200


#Using `links` within the MyJohnDeere API#

The MyJohnDeere API uses [HATEOAS](https://en.wikipedia.org/wiki/HATEOAS) links. Now that we've called the `API Catalog` lets explore the catalog by following the links.

Adding a helper method can make links easier to use. The following functions convert from the JSON link object into a python dictionary. Now the API Catalog is a little easier to work with.

In [7]:
def convert_links_array_to_dictionary(links_a):
    link_dict = dict()
    for link_o in links_a:
        key = link_o['rel']
        value = link_o['uri']
        link_dict[key] = value
    return link_dict
api_catalog_links_as_dictionary = convert_links_array_to_dictionary(api_catalog_response.json()['links'])
api_catalog_links_as_dictionary

def replace_links_as_object_array_with_links_as_dictionary(object_with_links_to_convert):
  object_with_links_converted = object_with_links_to_convert
  object_with_links_converted['links'] = convert_links_array_to_dictionary(object_with_links_to_convert['links'])
  return object_with_links_converted

api_catalog_with_clean_links = replace_links_as_object_array_with_links_as_dictionary(api_catalog_response.json())
api_catalog_with_clean_links

{'@type': 'ApiCatalog',
 'links': {'files': 'https://sandboxapi.deere.com/platform/files',
  'fileTransfers': 'https://sandboxapi.deere.com/platform/fileTransfers',
  'currentUser': 'https://sandboxapi.deere.com/platform/users/@currentUser',
  'organizations': 'https://sandboxapi.deere.com/platform/organizations',
  'partnerships': 'https://sandboxapi.deere.com/platform/partnerships',
  'lastKnownEngineHours': 'https://sandboxapi.deere.com/platform/engineHours?lastKnown=true',
  'geofencesCreate': 'https://sandboxapi.deere.com/platform/organizations/{orgId}/geofences',
  'geofencesListGet': 'https://sandboxapi.deere.com/platform/organizations/{orgId}/geofences',
  'geofencesUpdate': 'https://sandboxapi.deere.com/platform/organizations/{orgId}/geofences/{geofenceId}',
  'geofencesDelete': 'https://sandboxapi.deere.com/platform/organizations/{orgId}/geofences/{geofenceId}',
  'hoursOfOperation': 'https://sandboxapi.deere.com/platform/hoursOfOperation',
  'machines': 'https://sandboxapi.d

That looks easier to work with. The `rel` has been converted to an attribute and the `uri` is now the value. Let us take it a step farther and modify the object itself to make using the responses themselves a bit easier.

When calling the MyJohnDeere API for resources, you can control which links can appear in the response. This is done via the `showLinks` request param, where the values act as a filter to on the returned links. Let's look at the `API Catalog` again, but with fewer links.

*Extra tip:* Not every link will be present; links only appear for API endpoints that you have permission to use. If your user context changes or you switch organizations, the available links may change.

In [8]:
api_catalog_showLinks_request_param = {'showLinks': 'currentToken,currentUser,organizations'}

api_catalog_response_with_request_param = oauth2_session.get(API_CATALOG_URI, headers = MYJOHNDEERE_V3_JSON_HEADERS, params = api_catalog_showLinks_request_param)
api_catalog_response_with_request_param.json()

{'@type': 'ApiCatalog',
 'links': [{'@type': 'Link',
   'rel': 'currentUser',
   'uri': 'https://sandboxapi.deere.com/platform/users/@currentUser'},
  {'@type': 'Link',
   'rel': 'organizations',
   'uri': 'https://sandboxapi.deere.com/platform/organizations'}]}

Using the `showLinks` request parameter is useful once you know which links your application needs. Fewer links will result in faster API responses.

Now lets chain our functions together to get just the links we want.

In [9]:
def create_show_links_request_param(links_to_show_comma_separated = None):
  return {'showLinks' : links_to_show_comma_separated}

def get_myjohndeere_api_json_response(oauth_session, myjohndeere_uri, headers = MYJOHNDEERE_V3_JSON_HEADERS, params = None):
  json_response = oauth2_session.get(myjohndeere_uri, headers = headers, params = params).json()
  return replace_links_as_object_array_with_links_as_dictionary(json_response)

api_catalog_response_with_clean_links = get_myjohndeere_api_json_response(oauth2_session, myjohndeere_uri = API_CATALOG_URI, params = create_show_links_request_param('currentToken,currentUser,organizations'))
api_catalog_response_with_clean_links

{'@type': 'ApiCatalog',
 'links': {'currentUser': 'https://sandboxapi.deere.com/platform/users/@currentUser',
  'organizations': 'https://sandboxapi.deere.com/platform/organizations'}}

Now that we can manage links, try making additional API calls by following the links. Let's follow the '`currentUser`' link to see more information about the currently logged in user.

In [10]:
current_user_link = api_catalog_response_with_clean_links['links']['currentUser']
current_user_response = get_myjohndeere_api_json_response(oauth2_session, myjohndeere_uri = current_user_link)
current_user_response

{'@type': 'User',
 'accountName': 'ChristianKau',
 'givenName': 'Christian',
 'familyName': 'Kau',
 'userType': 'Customer',
 'links': {'self': 'https://sandboxapi.deere.com/platform/users/ChristianKau',
  'organizations': 'https://sandboxapi.deere.com/platform/users/ChristianKau/organizations',
  'emailAddresses': 'https://sandboxapi.deere.com/platform/users/ChristianKau/emailAddresses'}}

#Working with the `Collection` object : counts, nextPage links, and combining results#

Many of the MyJohnDeere API's return collections of objects, as compared to the single object responses we've seen with the `API Catalog` and `currentUser` endpoints. These collections are wrapped in a JSON object that contains metadata to support pagination. You can see an example by following the '`organizations`' link from the `currentUser`

In [11]:
organizations_for_current_user_link = current_user_response['links']['organizations']
get_myjohndeere_api_json_response(oauth2_session, myjohndeere_uri = organizations_for_current_user_link)

{'links': {'self': 'https://sandboxapi.deere.com/platform/users/ChristianKau/organizations'},
 'total': 1,
 'values': [{'@type': 'Organization',
   'name': 'Christian Kau',
   'type': 'customer',
   'member': True,
   'internal': False,
   'id': '2469981',
   'links': [{'@type': 'Link',
     'rel': 'self',
     'uri': 'https://sandboxapi.deere.com/platform/organizations/2469981'},
    {'@type': 'Link',
     'rel': 'preferences',
     'uri': 'https://sandboxapi.deere.com/platform/organizations/2469981/preferences'},
    {'@type': 'Link',
     'rel': 'manage_connection',
     'uri': 'https://connections.deere.com/connections/0oa9u3fohqdKLFBbs5d7/connections-dialog?orgId=2469981'},
    {'@type': 'Link',
     'rel': 'machines',
     'uri': 'https://sandboxapi.deere.com/platform/organizations/2469981/machines'},
    {'@type': 'Link',
     'rel': 'organizationMaintenancePlans',
     'uri': 'https://sandboxapi.deere.com/platform/organizations/2469981/maintenancePlans'},
    {'@type': 'Link',


The collection wrapper contains links for '`self`', '`nextPage`', and '`previousPage`'. There is also a `total` attribute that represents how many objects exist within the collection across all pages, and a `values` attribute that contains the actual collection of objects. 


In [12]:
def get_myjohndeere_api_collection_json_response(oauth_session, myjohndeere_uri, headers = MYJOHNDEERE_V3_JSON_HEADERS, params = None):
  collection_json_response = get_myjohndeere_api_json_response(oauth_session, myjohndeere_uri, headers, params)
  values_from_collection = collection_json_response['values']
  values_to_add_back_to_collection = []
  for object in values_from_collection:
    values_to_add_back_to_collection.append(replace_links_as_object_array_with_links_as_dictionary(object))
  collection_json_response['values'] = values_to_add_back_to_collection
  return collection_json_response

In [13]:
organizations_for_current_user_link = current_user_response['links']['organizations']
get_myjohndeere_api_collection_json_response(oauth2_session, myjohndeere_uri = organizations_for_current_user_link)

{'links': {'self': 'https://sandboxapi.deere.com/platform/users/ChristianKau/organizations'},
 'total': 1,
 'values': [{'@type': 'Organization',
   'name': 'Christian Kau',
   'type': 'customer',
   'member': True,
   'internal': False,
   'id': '2469981',
   'links': {'self': 'https://sandboxapi.deere.com/platform/organizations/2469981',
    'preferences': 'https://sandboxapi.deere.com/platform/organizations/2469981/preferences',
    'manage_connection': 'https://connections.deere.com/connections/0oa9u3fohqdKLFBbs5d7/connections-dialog?orgId=2469981',
    'machines': 'https://sandboxapi.deere.com/platform/organizations/2469981/machines',
    'organizationMaintenancePlans': 'https://sandboxapi.deere.com/platform/organizations/2469981/maintenancePlans',
    'chemicals': 'https://sandboxapi.deere.com/platform/organizations/2469981/chemicals',
    'varieties': 'https://sandboxapi.deere.com/platform/organizations/2469981/varieties',
    'fertilizers': 'https://sandboxapi.deere.com/platform

Now that we have simplified the JSON that we're working with, we will next handle pagination. The MyJohnDeere API uses [pagination](https://developer.deere.com/#!help&doc=.%2Fgetstarted%2FPaginationGuide.htm&anchor=) in order to reduce the load per API request. 

In order to control pagination, MyJohnDeere API's use matrix parameters `start` and `count`. To demonstrate, set the `count` to one.

In [14]:
organizations_for_current_user_link = current_user_response['links']['organizations']
organizations_for_current_user_link_with_count_of_1 = organizations_for_current_user_link + ';count=1'
get_myjohndeere_api_collection_json_response(oauth2_session, myjohndeere_uri = organizations_for_current_user_link_with_count_of_1)

{'links': {'self': 'https://sandboxapi.deere.com/platform/users/ChristianKau/organizations;count=1'},
 'total': 1,
 'values': [{'@type': 'Organization',
   'name': 'Christian Kau',
   'type': 'customer',
   'member': True,
   'internal': False,
   'id': '2469981',
   'links': {'self': 'https://sandboxapi.deere.com/platform/organizations/2469981',
    'preferences': 'https://sandboxapi.deere.com/platform/organizations/2469981/preferences',
    'manage_connection': 'https://connections.deere.com/connections/0oa9u3fohqdKLFBbs5d7/connections-dialog?orgId=2469981',
    'machines': 'https://sandboxapi.deere.com/platform/organizations/2469981/machines',
    'organizationMaintenancePlans': 'https://sandboxapi.deere.com/platform/organizations/2469981/maintenancePlans',
    'chemicals': 'https://sandboxapi.deere.com/platform/organizations/2469981/chemicals',
    'varieties': 'https://sandboxapi.deere.com/platform/organizations/2469981/varieties',
    'fertilizers': 'https://sandboxapi.deere.com/

Assuming the current user is a member of two organizations, we now see a `nextPage` link which has `start=1` and `count=1`. Let's enhance our last method to follow the `nextPage` link.

In [15]:
def get_myjohndeere_api_collection_values_for_all_pages(oauth_session, myjohndeere_uri, headers = MYJOHNDEERE_V3_JSON_HEADERS, params = None):
  collection_response = get_myjohndeere_api_collection_json_response(oauth_session, myjohndeere_uri, headers, params)
  collection_links = collection_response['links']
  while 'nextPage' in collection_links:
    next_page_response = get_myjohndeere_api_collection_json_response(oauth_session, collection_links['nextPage'], headers, params)
    collection_links = next_page_response['links']
    collection_response['values'].extend(next_page_response['values'])
  return collection_response['values']

In [16]:
organizations_for_current_user_link = current_user_response['links']['organizations']
organizations_for_current_user_link_with_count_of_1 = organizations_for_current_user_link + ';count=1'
get_myjohndeere_api_collection_values_for_all_pages(oauth2_session, myjohndeere_uri = organizations_for_current_user_link_with_count_of_1)

[{'@type': 'Organization',
  'name': 'Christian Kau',
  'type': 'customer',
  'member': True,
  'internal': False,
  'id': '2469981',
  'links': {'self': 'https://sandboxapi.deere.com/platform/organizations/2469981',
   'preferences': 'https://sandboxapi.deere.com/platform/organizations/2469981/preferences',
   'manage_connection': 'https://connections.deere.com/connections/0oa9u3fohqdKLFBbs5d7/connections-dialog?orgId=2469981',
   'machines': 'https://sandboxapi.deere.com/platform/organizations/2469981/machines',
   'organizationMaintenancePlans': 'https://sandboxapi.deere.com/platform/organizations/2469981/maintenancePlans',
   'chemicals': 'https://sandboxapi.deere.com/platform/organizations/2469981/chemicals',
   'varieties': 'https://sandboxapi.deere.com/platform/organizations/2469981/varieties',
   'fertilizers': 'https://sandboxapi.deere.com/platform/organizations/2469981/fertilizers',
   'machineStylePreferences': 'https://sandboxapi.deere.com/platform/organizations/2469981/mac

To be more realistic, add another helper function to set the pagination count to 100. This is the maximum page size.

In [17]:
def default_to_count_100(followable_link) :
  return followable_link + ";count=100"

To bring all of this together, get the organization list with a page size of 100. Add a `showLinks` request param to limit links to just the `machines`, `files`, and `fields` links. 

*Extra tip: * If you don't want pagination, you can disable it by adding a header named `No_Paging` with value of `true`. For some endpoints you can also disable links by adding a query param `showLinks=none`.

In [18]:
show_links_request_param_for_organization_list = {'showLinks' : 'machines,files,fields'}

organizations_for_current_user_link = current_user_response['links']['organizations']
organizations_for_current_user_link_with_page_size_of_100 = default_to_count_100(organizations_for_current_user_link)
get_myjohndeere_api_collection_values_for_all_pages(oauth2_session, organizations_for_current_user_link_with_page_size_of_100, params = show_links_request_param_for_organization_list)

[{'@type': 'Organization',
  'name': 'Christian Kau',
  'type': 'customer',
  'member': True,
  'internal': False,
  'id': '2469981',
  'links': {'machines': 'https://sandboxapi.deere.com/platform/organizations/2469981/machines',
   'files': 'https://sandboxapi.deere.com/platform/organizations/2469981/files',
   'fields': 'https://sandboxapi.deere.com/platform/organizations/2469981/fields'}}]

#Using `embed` to optimize data retrieval#

To replace multiple API calls to retrieve several related objects, the MyJohnDeere API's support the concept of an '`embed`'. Instead of retrieving a list of fields and then making a series of API calls to get the associated `client` and `farm` for each field, you can request that clients and farms are embedded in the initial fields call. Then each `field` in the fields list will have pre-populated `client` and `farm` attributes.

Let's start by finding an organization that has a `fields` link.

In [19]:
show_field_links_request_param_for_organization_list = {'showLinks' : 'fields'}

organization_list_for_logged_in_user = get_myjohndeere_api_collection_values_for_all_pages(oauth2_session, organizations_for_current_user_link_with_page_size_of_100, params = show_field_links_request_param_for_organization_list)
organization_which_has_fields_link = None
for organization in organization_list_for_logged_in_user:
  if 'fields' in organization['links']:
    organization_which_has_fields_link = organization
    break
    
assert(organization_which_has_fields_link != None), "The logged in user is not a staff member of any organization in which they can see fields"
organization_which_has_fields_link

{'@type': 'Organization',
 'name': 'Christian Kau',
 'type': 'customer',
 'member': True,
 'internal': False,
 'id': '2469981',
 'links': {'fields': 'https://sandboxapi.deere.com/platform/organizations/2469981/fields'}}

Let's follow the `fields` link to see which fields are available to our user within this organization.

In [20]:
organization_field_list_uri = organization_which_has_fields_link['links']['fields']
organization_field_list_uri_with_count_100 = default_to_count_100(organization_field_list_uri)


field_list_for_organization = get_myjohndeere_api_collection_values_for_all_pages(oauth2_session, organization_field_list_uri_with_count_100)
field_list_for_organization

[{'@type': 'Field',
  'name': '22sep sp run2',
  'lastModifiedTime': '1970-01-01T00:00:00.000Z',
  'archived': False,
  'id': 'f9b7198d-a8b7-4f87-a4d2-d42c00b5fbc2',
  'links': {'self': 'https://sandboxapi.deere.com/platform/organizations/2469981/fields/f9b7198d-a8b7-4f87-a4d2-d42c00b5fbc2',
   'clients': 'https://sandboxapi.deere.com/platform/organizations/2469981/fields/f9b7198d-a8b7-4f87-a4d2-d42c00b5fbc2/clients',
   'farms': 'https://sandboxapi.deere.com/platform/organizations/2469981/fields/f9b7198d-a8b7-4f87-a4d2-d42c00b5fbc2/farms',
   'owningOrganization': 'https://sandboxapi.deere.com/platform/organizations/2469981',
   'boundaries': 'https://sandboxapi.deere.com/platform/organizations/2469981/fields/f9b7198d-a8b7-4f87-a4d2-d42c00b5fbc2/boundaries',
   'simplifiedBoundaries': 'https://sandboxapi.deere.com/platform/organizations/2469981/fields/f9b7198d-a8b7-4f87-a4d2-d42c00b5fbc2/boundaries?simple=true',
   'addBoundary': 'https://sandboxapi.deere.com/platform/organizations/24

As long as your organization has at least one field, you should see the `field` object with a variety of links. We could follow the links to retrieve the associated `client`, `farm`, and `boundary`, but it's faster to get everything in one API call. 

Let's make the same call as before, but use the `embed` request param to get more information.

In [21]:
farm_client_activeBoundary_embed_request_param = {'embed' : 'farms,clients,activeBoundary'}

field_list_for_organization_with_embed = get_myjohndeere_api_collection_values_for_all_pages(oauth2_session, organization_field_list_uri_with_count_100, params = farm_client_activeBoundary_embed_request_param)
field_list_for_organization_with_embed

[{'@type': 'Field',
  'name': '22sep sp run2',
  'farms': {'@type': 'Farms',
   'farms': [{'@type': 'Farm',
     'name': '---',
     'archived': False,
     'clientUri': 'https://sandboxapi.deere.com/platform/organizations/2469981/clients/',
     'id': '48758010-7e48-4e59-8a2a-b776c975fa2b',
     'links': [{'@type': 'Link',
       'rel': 'self',
       'uri': 'https://sandboxapi.deere.com/platform/organizations/2469981/farms/48758010-7e48-4e59-8a2a-b776c975fa2b'},
      {'@type': 'Link',
       'rel': 'fields',
       'uri': 'https://sandboxapi.deere.com/platform/organizations/2469981/farms/48758010-7e48-4e59-8a2a-b776c975fa2b/fields'},
      {'@type': 'Link',
       'rel': 'clients',
       'uri': 'https://sandboxapi.deere.com/platform/organizations/2469981/farms/48758010-7e48-4e59-8a2a-b776c975fa2b/clients'},
      {'@type': 'Link',
       'rel': 'owningOrganization',
       'uri': 'https://sandboxapi.deere.com/platform/organizations/2469981'}]}],
   'otherAttributes': {}},
  'client

We strongly encourage our Connected Software Companies to use `embed` on frequently-used API requests. If you find an API that you wish had an `embed`, please let John Deere API support know.

#Using Deere ETags to monitor for changes#

[Deere ETags](https://developer-portal.deere.com/#/start-developing/help/get-started/help-deere-tags) is a custom implementation of ETags. It helps keep data in sync between your system and ours by telling you what changed since the last time you made an API call.


In [ ]:
def create_myjohndeere_api_header_with_deere_etag(deere_etag_value = 'nil'):
  return {'Accept': 'application/vnd.deere.axiom.v3+json', 'x-deere-signature' : deere_etag_value}

In [ ]:
deere_etag_value = 'nil'

json_plus_deere_etag_headers = create_myjohndeere_api_header_with_deere_etag(deere_etag_value)

field_list_response_with_deere_etag = oauth2_session.get(organization_field_list_uri_with_count_100, 
                                                        params = farm_client_activeBoundary_embed_request_param,
                                                        headers = json_plus_deere_etag_headers)
field_list_response_with_deere_etag.json()

The response looks the same, except for the new `x-deere-signature` header. It contains a UUID that can be used next time we make this API call.

In [ ]:
field_list_response_with_deere_etag.headers['x-deere-signature']

We can use this UUID in future requests to see if anything has changed. Let's copy the `x-deere-signature` guid into a new request. 

*Extra tip:* This UUID is only valid for a single URL. For each combination of path parameters, you'll need to store a separate ETag UUID.

In [ ]:
header_with_deere_etag_value = create_myjohndeere_api_header_with_deere_etag(field_list_response_with_deere_etag.headers['x-deere-signature'])

In [ ]:
field_list_response_with_deere_etag_using_etag = oauth2_session.get(organization_field_list_uri_with_count_100, 
                                                        params = farm_client_activeBoundary_embed_request_param,
                                                        headers = header_with_deere_etag_value)
field_list_response_with_deere_etag_using_etag

In [ ]:
if field_list_response_with_deere_etag_using_etag.status_code == 200:
  #If a 200 response is seen, your own application would need to update your downloaded field list. This statement instead just updates the x-deere-signature to a new value
  header_with_deere_etag_value = create_myjohndeere_api_header_with_deere_etag(field_list_response_with_deere_etag_using_etag.headers['x-deere-signature'])

#Optimizing your calls with GZip#

You can reduce network traffic by asking the API's to compress their responses. The MyJohnDeere API's support GZip compression.

Let's add an `Accept-Encoding=GZip` header and make some more API calls.

In [22]:
MYJOHNDEERE_V3_JSON_AND_GZIP_HEADERS = {'Accept': 'application/vnd.deere.axiom.v3+json', 'Content-Type': 'application/vnd.deere.axiom.v3+json', 'Accept-Encoding' : 'gzip' }

organization_field_list_with_embed_responses_gzip = oauth2_session.get(organization_field_list_uri_with_count_100, headers = MYJOHNDEERE_V3_JSON_AND_GZIP_HEADERS, params = farm_client_activeBoundary_embed_request_param)
field_list_response_size_in_byte_with_gzip = len(organization_field_list_with_embed_responses_gzip.content)
print("Response size of of the field list with GZip was " + str(field_list_response_size_in_byte_with_gzip))

Response size of of the field list with GZip was 43940


#Recommended administration features to add to your integration#

###Using the `currentUser` object as a 'foreign key' into the MyJohnDeere API###

Remember that user accounts in your system may be set up differently than user accounts in the John Deere system. Especially with large farming organizations, it is useful to be able to understand who a user is within John Deere Operations Center. We'll use the 'currentUser' link from the API Catalog to discover some key details about the logged in user.

In [23]:
current_user_uri = api_catalog_response_with_clean_links['links']['currentUser']
current_user_json_response = get_myjohndeere_api_json_response(oauth2_session, current_user_uri)
current_user_json_response

{'@type': 'User',
 'accountName': 'ChristianKau',
 'givenName': 'Christian',
 'familyName': 'Kau',
 'userType': 'Customer',
 'links': {'self': 'https://sandboxapi.deere.com/platform/users/ChristianKau',
  'organizations': 'https://sandboxapi.deere.com/platform/users/ChristianKau/organizations',
  'emailAddresses': 'https://sandboxapi.deere.com/platform/users/ChristianKau/emailAddresses'}}

The `currentUser` object contains a few key elements that you'll likely find useful. The `userType` attribute lets you know if the logged in user is a John Deere customer or a John Deere dealer, which you can use to give a more customized experience. The `accountName` attribute is the username of the customer within the John Deere system. Your own tooling should keep a record of this whenever you encounter errors that you plan to submit to John Deere's [API support team](mailto:APIDevSupport@JohnDeere.com). This will help the support team help you fix issues quickly.

#Handling API errors#

###Dealing with 500 responses###

While we target high uptime, every web API has problems sometimes. We've documented [how to handle some of the most common errors](https://developer-portal.deere.com/#/start-developing/help/get-started/common-errors).

When you do encounter an error, please don't retry immediately - that makes things worse! We recommend that you use an exponential backoff loop. For this code section, we'll use [httpstat.us](https://httpstat.us/) as a demonstration tool to automatically generate 500 errors.

*Extra tip:* This demo simply doubles the wait time after each failure. Another good alternative (in milliseconds) is `Thread.Sleep((int) Math.Pow(10, retryCount));`.

In [26]:
uri_that_always_generates_500_errors = "https://httpstat.us/500"
oauth2_session.get(uri_that_always_generates_500_errors)

<Response [500]>

Now we'll add some code to retry when we encounter a 500. We will also import some libraries for dealing with time.

In [27]:
import time
import datetime

def is_status_code_a_server_error(http_status_code):
  return http_status_code >= 500 and http_status_code < 600

def print_current_time_to_console():
  print("Making a call at " + str(datetime.datetime.now()))

def call_api_with_500_retry_logic(oauth_session, uri_to_call):
  print_current_time_to_console()
  timer_in_seconds = 1
  api_response = oauth_session.get(uri_to_call)
  while(is_status_code_a_server_error(api_response.status_code) and timer_in_seconds < 30):
    time.sleep(timer_in_seconds)
    timer_in_seconds = timer_in_seconds * 2
    print_current_time_to_console()
    api_response = oauth_session.get(uri_to_call)
  print("Unable to succeed in the API call. This would be a good point to log the exact API call made, the username of the caller, as well as the date")

In [28]:
call_api_with_500_retry_logic(oauth2_session, uri_that_always_generates_500_errors)

Making a call at 2023-06-15 10:24:56.998281
Making a call at 2023-06-15 10:24:58.154603
Making a call at 2023-06-15 10:25:00.297535
Making a call at 2023-06-15 10:25:04.451837
Making a call at 2023-06-15 10:25:12.585395
Making a call at 2023-06-15 10:25:28.725043
Unable to succeed in the API call. This would be a good point to log the exact API call made, the username of the caller, as well as the date


###Dealing with 429 responses###

The MyJohnDeere API has [built-in flood protection](https://developer-portal.deere.com/#/start-developing/help/get-started/common-errors) that generates HTTP 429 responses. It is triggered by too many requests from a single application, or when the MyJohnDeere API server is under unusually high load.

As a demonstration, we'll set up a `mock` to simulate `429` responses from the `API Catalog`.



In [29]:
!pip install requests_mock

In [30]:
import requests
import requests_mock

mock_api_catalog_uri = 'mock://sandboxapi.deere.com/platform/'

session = requests.Session()
adapter = requests_mock.Adapter()
session.mount('mock', adapter)

retry_after_two_seconds_header = {'Retry-After' : '2'}
adapter.register_uri('GET', mock_api_catalog_uri, status_code = 429, headers = retry_after_two_seconds_header)

With the 429 response, the MyJohnDeere API server recommendations how long to wait - in seconds - before attempting to retry. We will use that response header in order to setup our sleep timers.

In [31]:
import time
import datetime

def is_status_code_429(http_status_code):
  return http_status_code == 429

def print_current_time_to_console():
  print("Making a call at " + str(datetime.datetime.now()))
  
def call_api_with_429_retry_logic(session, uri_to_call):
  print_current_time_to_console()
  attempts = 0
  api_response = session.get(uri_to_call)
  while(is_status_code_429(api_response.status_code) and attempts < 3):
    time.sleep(int(api_response.headers['Retry-After']))
    attempts = attempts + 1
    print_current_time_to_console()
    api_response = session.get(uri_to_call)
  print("For the 429, it can still be useful to have a maximum amount of retrys before considering the API call a failure")

In [32]:
call_api_with_429_retry_logic(session, mock_api_catalog_uri)

Making a call at 2023-06-15 10:26:12.039263
Making a call at 2023-06-15 10:26:14.043463
Making a call at 2023-06-15 10:26:16.057694
Making a call at 2023-06-15 10:26:18.072528
For the 429, it can still be useful to have a maximum amount of retrys before considering the API call a failure


#Selecting the right organization for a grower#

For most applications connected to the MyJohnDeere API, your end user is a farmer who is unlikely to be a member of multiple organizations in Operations Center. If a user is a member of multiple organizations, you will need to select which one to use. In the second cell, make sure to change the `selected_organization_index` to an organization from the results from the first cell.

In [33]:
current_user_response = get_myjohndeere_api_json_response(oauth2_session, current_user_uri)
organizations_for_current_user_uri = current_user_response['links']['organizations']
organizations_for_current_user_uri = default_to_count_100(organizations_for_current_user_uri)
organizations_for_current_user = get_myjohndeere_api_collection_values_for_all_pages(oauth2_session, organizations_for_current_user_uri)
organization_index = 0
for organization in organizations_for_current_user:
  print("[" + str(organization_index) + "] " + organization['name'])
  organization_index = organization_index + 1

[0] Christian Kau


In [34]:
#Replace the value of selected_organization_index to one of the values listed from the cell above
selected_organization_index = 0

selected_organization = organizations_for_current_user[selected_organization_index]
print("The organization chosen has a name of \"" + selected_organization['name'] + "\" and an id of " + selected_organization['id'])

The organization chosen has a name of "Christian Kau" and an id of 2469981


#Selecting the right organization for a contractor or a John Deere dealer#

Applications that target contractors or John Deere dealers often need to manage organizations differently. It is common for a contractor or dealer to support a large number of organizations, and your application will need a way to let the user select which one to use.

Note that in this case you want to keep data separated by organization. Merging data across organizations is probably not what a dealer or contractor wants.

In [35]:
api_catalog_response = get_myjohndeere_api_json_response(oauth2_session, API_CATALOG_URI)
organizations_uri = api_catalog_response['links']['organizations']
organizations_uri = default_to_count_100(organizations_uri)

organizations_visible_to_logged_in_user = get_myjohndeere_api_collection_values_for_all_pages(oauth2_session, organizations_uri)
organization_index = 0
for organization in organizations_visible_to_logged_in_user:
  print("[" + str(organization_index) + "] " + organization['name'])
  organization_index = organization_index + 1

[0] Christian Kau


In [37]:
#Replace the values in the array below with the organizations you want to pick from above
selected_organizations_array = [0,]

selected_organizations = [organizations_visible_to_logged_in_user[i] for i in selected_organizations_array]
print("The number of organizations chosen was " + str(len(selected_organizations)))

The number of organizations chosen was 1


#Getting your application to production#

###Application access while in Sandbox###

Applications on developer.deere.com have two lifecycle phases. In "Sandbox", your application is only available to your application team. In "Production", real customers can use your application. This is the only major difference between the two - in both phases, your application calls the production MyJohnDeere backend.

This is a picture showing the "Team Members" of an application on developer.deere.com. Only "Team" members of a `Sandbox` application on developer.deere.com are allowed to authenticate. You can invite additional team members as needed for development and testing.

![Team Section on developer.deere.com](https://github.com/JohnDeere/DevelopWithDeereNotebooks/raw/master/Best%20Practices/TeamOnDeveloperDeereCom.png)

###Getting to production###

When your application is ready to go to production with the MyJohnDeere API, it must pass two checks before we will grant Production access.

Schedule a review with John Deere's API support team. You do not need to share your source code - we just want to check our logs. Your application should be used like a real customer would use your application. This can include initial signup, creating data, and polling the MyJohnDeere API's.

If anything critical is noticed in the review, such as potential DDoS attacks or inefficient API usage, we'll make suggestions.

The John Deere Business Development team needs a signed API usage agreement. This agreement goes over all of the APIs required, the usage limits for those APIs, and allowed behaviors.

Once both steps are complete, production access will be granted. At that time, change the base URI you use to call MyJohnDeere from 'https://sandboxapi.deere.com/platform/' to 'https://partnerapi.deere.com/platform/'.